# **Currency Tracker**
Keeping track of currency deposited by means of multiple classifiers

### **Importing necessary libraries**

In [1]:
import tensorflow as tf
from keras.models import load_model
import cv2
import numpy as np
import datetime

2023-12-01 13:43:16.746663: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 13:43:16.746704: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 13:43:16.746741: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 13:43:16.754332: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### **Set initial accounts to 0**

In [2]:
accounts = 0

### **Loading models**

In [3]:
binary_classification_model_path = "saved-models/binary-classifier.h5"
notes_classification_model_path = "saved-models/notes-classifier.h5"
coins_classification_model_path = "saved-models/coins-classifier.h5"

In [4]:
binary_classifier = load_model(binary_classification_model_path)
notes_classifier = load_model(notes_classification_model_path)
coins_classifier = load_model(coins_classification_model_path)

2023-12-01 13:43:23.817707: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-01 13:43:23.823122: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-01 13:43:23.823528: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

### **Classification**

In [5]:
coins_classes = ['1', '2', '5', '10']
notes_classes = ['10', '20', '50', '100', '200', '500', '2000']

In [12]:
image_path = 'data/multiclass/notes/50/203.png'
frame = cv2.imread(image_path)
frame = cv2.resize(frame, (320, 170))
frame = frame / 255.0

In [13]:
binary_prediction = binary_classifier.predict(np.expand_dims(frame, axis=0))
binary_class = np.argmax(binary_prediction)
binary_probability = np.max(binary_prediction)

1/1 [==============================] - 0s 17ms/step


In [14]:
print(f"Binary Classification: {['Note', 'Coin'][binary_class]} - Probability: {binary_probability}")

Binary Classification: Note - Probability: 0.9999955892562866


In [15]:
if binary_class == 0:  # If it's a note, use notes_classifier
    notes_prediction = notes_classifier.predict(np.expand_dims(frame, axis=0))
    notes_class = np.argmax(notes_prediction)
    notes_probability = np.max(notes_prediction)
    num = int(notes_classes[notes_class])
    accounts += int(notes_classes[notes_class])
    # Print the notes classification result
    print(f"Notes Classification: {notes_classes[notes_class]} - Probability: {notes_probability}")

else:  # If it's a coin, use coins_classifier
    coins_prediction = coins_classifier.predict(np.expand_dims(frame, axis=0))
    coins_class = np.argmax(coins_prediction)
    coins_probability = np.max(coins_prediction)
    num = int(coins_classes[coins_class])
    accounts += int(coins_classes[coins_class])
    # Print the coins classification result
    print(f"Coins Classification: {coins_classes[coins_class]} - Probability: {coins_probability}")

1/1 [==============================] - 0s 22ms/step
Notes Classification: 50 - Probability: 0.949052095413208


In [16]:
print(accounts)

100


### **Writing current accounts to accounts.txt, along with timestamp**

In [17]:
current_datetime = datetime.datetime.now()
formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
data_to_write = f"{formatted_datetime} --- Deposited amount is: {num} --- Total amount is: {accounts}\n"
file_path = 'accounts.txt'

with open(file_path, 'a') as file:
    file.write(data_to_write)

print(f"Data written to {file_path}")

Data written to accounts.txt
